## Tutorial 5: Position- and distance-based connectivity
* http://doc.netpyne.org/tutorial.html#tutorial-5-position-and-distance-based-connectivity

* implement weights, probabilities of connection, and delays that depend on cell positions or distances.

* We will build a cortical-like network with six populations (three excitatory and three inhibitory) distributed in three layers: 2/3, 4 and 5.

* By convention we take the X and Z to be the horizontal or lateral dimensions, and Y to be the vertical dimension (representing cortical depth in this case). To define a cuboid with volume of 100x1000x100 um (i.e. horizontal spread of 100x100 um and cortical depth of 1000um) 
* note that Z is not vertical depth.

In [9]:
# load library
from netpyne import specs, sim

# Network parameters
netParams = specs.NetParams()

In [10]:
# spatial definition of model box
netParams.sizeX = 100  # x-dimension (horizontal length) size in um
netParams.sizeY = 1000  # y-dimension (vertical height or cortical depth) size in um
netParams.sizeZ = 100  # z-dimension (horizontal length) size in um
netParams.propVelocity = 100.0  # propagation velocity (um/ms)
netParams.probLengthConst = 150.0  # length constant for conn probability (um)

In [11]:
## Cell types
secs = {}  # sections dict
secs["soma"] = {"geom": {}, "mechs": {}}  # soma params dict
secs["soma"]["geom"] = {"diam": 15, "L": 14, "Ra": 120.0}  # soma geometry
secs["soma"]["mechs"]["hh"] = {
    "gnabar": 0.13,
    "gkbar": 0.036,
    "gl": 0.003,
    "el": -70,
}  # soma hh mechanism
netParams.cellParams["E"] = {"secs": secs}  # add dict to list of cell params

secs = {}  # sections dict
secs["soma"] = {"geom": {}, "mechs": {}}  # soma params dict
secs["soma"]["geom"] = {"diam": 10.0, "L": 9.0, "Ra": 110.0}  # soma geometry
secs["soma"]["mechs"]["hh"] = {
    "gnabar": 0.11,
    "gkbar": 0.036,
    "gl": 0.003,
    "el": -70,
}  # soma hh mechanism
netParams.cellParams["I"] = {"secs": secs}  # add dict to list of cell params

In [12]:
## Population parameters
netParams.popParams["E2"] = {
    "cellType": "E",
    "numCells": 50,
    "yRange": [100, 300],
}  # layer2/3 depth
netParams.popParams["I2"] = {"cellType": "I", "numCells": 50, "yRange": [100, 300]}
netParams.popParams["E4"] = {
    "cellType": "E",
    "numCells": 50,
    "yRange": [300, 600],
}  # layer4 depth
netParams.popParams["I4"] = {"cellType": "I", "numCells": 50, "yRange": [300, 600]}
netParams.popParams["E5"] = {
    "cellType": "E",
    "numCells": 50,
    "ynormRange": [0.6, 1.0],
}  # layer5 depth
netParams.popParams["I5"] = {"cellType": "I", "numCells": 50, "ynormRange": [0.6, 1.0]}

In [13]:
## Synaptic mechanism parameters
netParams.synMechParams["exc"] = {
    "mod": "Exp2Syn",
    "tau1": 0.8,
    "tau2": 5.3,
    "e": 0,
}  # NMDA synaptic mechanism
netParams.synMechParams["inh"] = {
    "mod": "Exp2Syn",
    "tau1": 0.6,
    "tau2": 8.5,
    "e": -75,
}  # GABA synaptic mechanism

In [14]:
# Stimulation parameters
netParams.stimSourceParams["bkg"] = {"type": "NetStim", "rate": 20, "noise": 0.3}
netParams.stimTargetParams["bkg->all"] = {
    "source": "bkg",
    "conds": {"cellType": ["E", "I"]},
    "weight": 0.01,
    # delay Gaussian distribution with mean 5 ms and standard deviation 2,
    # and have a minimum value of 1 ms.
    "delay": "max(1, normal(5,2))",
    "synMech": "exc",
}

In [15]:
# Simulation options
simConfig = (
    specs.SimConfig()
)  # object of class SimConfig to store simulation configuration

simConfig.duration = 1 * 1e3  # Duration of the simulation, in ms
simConfig.dt = 0.05  # Internal integration timestep to use
simConfig.verbose = False  # Show detailed messages
simConfig.recordTraces = {
    "V_soma": {"sec": "soma", "loc": 0.5, "var": "v"}
}  # Dict with traces to record
simConfig.recordStep = 1  # Step size in ms to save data (e.g. V traces, LFP, etc)
simConfig.filename = "./results/Tutorial05/tut5"  # Set file output name
simConfig.savePickle = False  # Save params, network and sim output to pickle file

simConfig.analysis["plotRaster"] = {
    "orderBy": "y",
    "orderInverse": True,
    "saveFig": True,
}  # Plot a raster
simConfig.analysis["plotTraces"] = {
    "include": [("E2", 0), ("E4", 0), ("E5", 5)],
    "saveFig": True,
}  # Plot recorded traces for this list of cells
simConfig.analysis["plot2Dnet"] = {
    "saveFig": True
}  # plot 2D cell positions and connections
simConfig.analysis["plotConn"] = {"saveFig": True}  # plot connectivity matrix

# Create network and run simulation
sim.createSimulateAnalyze(netParams=netParams, simConfig=simConfig)


Start time:  2024-10-29 11:54:21.356545




Creating network of 6 cell populations on 1 hosts...: 100%|##########|

  Number of cells on node 0: 300 
  Done; cell creation time = 0.03 s.
Making connections...
  Number of connections on node 0: 0 
  Done; cell connection time = 0.00 s.
Adding stims...
  Number of stims on node 0: 300 
  Done; cell stims creation time = 0.03 s.
Recording 3 traces of 1 types on node 0

Running simulation using NEURON for 1000.0 ms...
  Done; run time = 1.13 s; real-time ratio: 0.89.

Gathering data...


  Done; gather time = 0.03 s.

Analyzing...
  Cells: 300
  Connections: 300 (1.00 per cell)
  Spikes: 6075 (20.25 Hz)
  Simulated time: 1.0 s; 1 workers
  Run time: 1.13 s
Preparing spike data...
Plotting raster...
Plotting recorded cell traces ... cell
Plotting 2D representation of network cell locations and connections...
Plotting connectivity matrix...
  Done; plotting time = 1.08 s

Total time = 2.31 s


In [16]:
# redefine the network, using spatial structure
netParams.connParams["E->all"] = {
    "preConds": {"cellType": "E"},
    "postConds": {"y": [100, 1000]},  #  E -> all (100-1000 um)
    "probability": 0.1,  # probability of connection
    "weight": "0.005*post_ynorm",  # synaptic weight
    "delay": "dist_3D/propVelocity",  # transmission delay (ms)
    "synMech": "exc",
}  # synaptic mechanism

In [17]:
simConfig.filename = "./results/Tutorial05/tut5_spatial"  # Set file output name
sim.createSimulateAnalyze(netParams=netParams, simConfig=simConfig)


Start time:  2024-10-29 11:57:45.723962




Creating network of 6 cell populations on 1 hosts...: 100%|##########|


  Number of cells on node 0: 300 
  Done; cell creation time = 0.04 s.
Making connections...


  E->all: 100%|##########| Creating synaptic connections for 300/300 postsynaptic cells on node 0 (probabilistic connectivity)


  Number of connections on node 0: 4539 
  Done; cell connection time = 0.57 s.
Adding stims...
  Number of stims on node 0: 300 
  Done; cell stims creation time = 0.03 s.
Recording 3 traces of 1 types on node 0

Running simulation using NEURON for 1000.0 ms...
  Done; run time = 3.74 s; real-time ratio: 0.27.

Gathering data...
  Done; gather time = 0.25 s.

Analyzing...
  Cells: 300
  Connections: 4839 (16.13 per cell)
  Spikes: 5157 (17.19 Hz)
  Simulated time: 1.0 s; 1 workers
  Run time: 3.74 s
Preparing spike data...
Plotting raster...
Plotting recorded cell traces ... cell
Plotting 2D representation of network cell locations and connections...
Plotting connectivity matrix...
  Done; plotting time = 3.84 s

Total time = 8.49 s


In [18]:
# use length constant for connection probability
netParams.connParams["I->E"] = {  # I -> E
    "preConds": {"cellType": "I"},  # presynaptic conditions
    "postConds": {"pop": ["E2", "E4", "E5"]},  # postsynaptic conditions
    "probability": "0.4*exp(-dist_3D/probLengthConst)",  # probability of connection
    "weight": 0.001,  # synaptic weight
    "delay": "dist_3D/propVelocity",  # transmission delay (ms)
    "synMech": "inh",
}  # synaptic mechanism

In [19]:
simConfig.filename = "./results/Tutorial05/tut5_lengCon"  # Set file output name
sim.createSimulateAnalyze(netParams=netParams, simConfig=simConfig)


Start time:  2024-10-29 12:00:24.483118




Creating network of 6 cell populations on 1 hosts...: 100%|##########|


  Number of cells on node 0: 300 
  Done; cell creation time = 0.03 s.
Making connections...


  E->all: 100%|##########| Creating synaptic connections for 300/300 postsynaptic cells on node 0 (probabilistic connectivity)
  I->E: 100%|##########| Creating synaptic connections for 150/150 postsynaptic cells on node 0 (probabilistic connectivity)


  Number of connections on node 0: 6796 
  Done; cell connection time = 0.92 s.
Adding stims...
  Number of stims on node 0: 300 
  Done; cell stims creation time = 0.03 s.
Recording 3 traces of 1 types on node 0

Running simulation using NEURON for 1000.0 ms...
  Done; run time = 5.70 s; real-time ratio: 0.18.

Gathering data...
  Done; gather time = 0.18 s.

Analyzing...
  Cells: 300
  Connections: 7096 (23.65 per cell)
  Spikes: 4778 (15.93 Hz)
  Simulated time: 1.0 s; 1 workers
  Run time: 5.70 s
Preparing spike data...
Plotting raster...
Plotting recorded cell traces ... cell
Plotting 2D representation of network cell locations and connections...
Plotting connectivity matrix...
  Done; plotting time = 5.49 s

Total time = 12.37 s
